## Steps for building waypoint graph for CARLA


### First Iteration
1. Generate original graph (version 0) from build_map_graph.py
2. Use graph_search.py to evaluate every points' path to goal point. It will store the points that cannot found the path to goal point using original version of graph
3. The points that cannot found the path to goal will be store in {Town}_not_found.txt


### Second Iteration
1. Run unit_test_usage.ipynb, and input "town name" and "each case in the {Town}_not_found.txt"
    - If path found:
        - Path will automatically add to the graph
    - If path not found:
        - (Option 1) Run manual_build.py to manually add new edge to the graph
        - (Option 2) If the distance between start and end is far, use manual_build.py to find the middle point, then use manual_add.py to find the path from "start to middle" and "middle to end"
2. Next
    - (Option 1) Keep Iterating through each testcase until solving every test case in {Town}_not_found.txt
    - (Option 2) Rerun graph_search.py to evaluate every points' path to goal point. Generate new {Town}_not_found.txt

In [ ]:
import networkx as nx
import numpy as np
import cv2
import json
import networkx as nx
import re
import argparse
import glob
import os

from route_generator import iter_all_white_points, draw_waypoint, rdp_algorithm, recorded_find_path
from add_path_to_graph import add_path, read_adjlist_with_tuples

##################### Input Town number #####################

town = int(input("Enter town number: 1, 2, 3, or 10"))

if town == 10:
    waypoint_map = cv2.imread("./waypoint_maps/Town10HD.png", 1)
    town_name = "Town10HD"
else:
    town_name = f"Town0{town}"
    waypoint_map = cv2.imread(f"./waypoint_maps/{town_name}.png", 1)


: 

In [10]:
##################### Read latest graph list of the given town #####################

list_of_files = glob.glob(f'graph_list/{town_name}/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)


if town == 10:
    index = int(latest_file[30:-8])
    new_file_name = latest_file[:30] + str(index+1) + latest_file[-8:]
else:
    index = int(latest_file[27:-8])
    new_file_name = latest_file[:27] + str(index+1) + latest_file[-8:]

G = read_adjlist_with_tuples(latest_file)


##################### Input line in the {town}_not_found.txt #####################

start_end_string = input("Enter each list with start-end point in the not found list")

# Example input:
# [(547, 645), (3871, 1346), 'Forward', '45_45_196']


entries = re.findall(r'\([^)]+\)', start_end_string)
start = entries[0]
start = start.strip('() ')
x0, y0 =  map(int, start.split(','))
first_tuple = tuple([x0, y0])

end = entries[1]
end = end.strip('() ')
x, y =  map(int, end.split(','))
second_tuple = tuple([x, y])

print(latest_file)
print(index)
print(first_tuple)
print(second_tuple)

IndexError: list index out of range

In [11]:
all_whites_pos = iter_all_white_points(waypoint_map)
start = first_tuple
end = second_tuple


new_img = waypoint_map.copy()
new_img[start[0]][start[1]] = np.array([0, 0, 255])
new_img[end[0]][end[1]] = np.array([0, 0, 255])

cv2.circle(new_img, start[::-1], radius=5, color=(0, 0, 255), thickness=5)
cv2.circle(new_img, end[::-1], radius=5, color=(255, 255, 0), thickness=5)


print("Start point:", start)
print("End point:", end)
print("Finding Path...")
# final_path = find_path(start, end, all_whites_pos)
flag, final_path = recorded_find_path(start, end, all_whites_pos, "", True)

##### Path not found #####
if flag == False:
    print("Not found!!!")
    print(final_path)
    for i, j in final_path[0]:
        new_img[i][j] = np.array([255, 255, 0])
    
    
    cv2.imwrite(f"test/not_found/test__{start}_{end}.png", new_img)
else:
    print("found!!!")
    print(final_path[0])
    print("Adding to the graph list...")
    
    add_path(G, final_path, new_file_name)
    print(new_file_name, "Store!")
    
    draw_img = draw_waypoint(waypoint_map, start, end, final_path[0])
    rdp_img, _ = rdp_algorithm(draw_img, final_path[0])
    cv2.imwrite(f"test/found/test__{start}_{end}.png", new_img)



Start point: (591, 2557)
End point: (4682, 3450)
Finding Path...


KeyboardInterrupt: 

In [7]:
############### Quick test for other start point with the same goal point

start, end = (2870, 1976), (3564, 2017)
fp = nx.shortest_path(G,  start,  end)
print(fp)


[(2870, 1976), (2874, 1978), (2904, 1991), (3006, 2010), (3112, 2014), (3220, 2014), (3328, 2015), (3436, 2016), (3544, 2017), (3564, 2017)]
